In [7]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
import sys
sys.path.append('/home/stevenrodas_snhu/Downloads')
import mainCRUD
from mainCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = 'aacuser'
password = 'steventestpassword'
host = 'nv-desktop-services.apporto.com'
port = 30818
db = 'AAC'
collection = 'animals'

# Connect to database via CRUD Module
db = mainCRUD.AnimalShelter(username, password, host, port, db, collection)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
#print(len(df.to_dict(orient='records')))
#print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'graziososalvare.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                style={
                    'height': '15%',
                    'width': '15%',
                    'float': 'right',
                    'position': 'relative',
                    'padding-top': 0,
                    'padding-right': 0
                    
                })
    ], href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        id='filter-container',
        className='row',
        style = {'margin': '25px', 'border': '1px solid #000', 'padding':'10px'},
        children=[
            html.H2('Filter', style={'text-align': 'center'}),
            html.Div(
                className='col',
                style={'display': 'flex', 'justify-content': 'center'},
                children=[
                    dcc.RadioItems(
                        id='filter-radio',
                        options=[
                            {'label': 'Water Rescue', 'value': 'Water'},
                            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain'},
                            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster'},
                            {'label': 'Reset', 'value': 'Reset'}
                        ],
                        value='',
                        labelStyle={'display': 'inline-block', 'margin-right': '10px'}
                    )
                ]
            
            )
        ]

    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,
                         sort_action="native",
                         filter_action="native",
                         style_table={'height': '300px', 'overflowY': 'auto'},
                         row_selectable="single",
                         
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Output('datable-id', 'columns')],
             [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    
    #updates dashboard based on filter from dropdown menu
    if filter_type == 'Water':
        df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog', 'breed': {'$in': ['Newfoundland']}}))
    elif filter_type == 'Mountain':
        df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog', 'breed': {'$in': ['Rottweiler']}}))
    elif filter_type == 'Disaster':
        df = pd.DataFrame.from_records(db.read({'animal_type': 'Dog', 'breed': {'$in': ['Doberman Pinscher']}}))
    else:
        df = pd.DataFrame.from_records(db.read({}))
        
    columns=[{'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns]
    data=df.to_dict('records')
    return (data, columns)



# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    df = pd.DataFrame(viewData)
    return [dcc.Graph(figure = px.pie(df, names='breed', title='Preferred Animals'))]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None or len(index) == 0:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:22952/
Empty DataFrame
Columns: []
Index: []
   rec_num age_upon_outcome animal_id animal_type                    breed  \
0        1          3 years   A746874         Cat   Domestic Shorthair Mix   
1        3          2 years   A716330         Dog  Chihuahua Shorthair Mix   
2        9          3 years   A720214         Dog   Labrador Retriever Mix   
3       10         3 months   A664290         Cat   Domestic Shorthair Mix   
4        2           1 year   A725717         Cat   Domestic Shorthair Mix   

          color date_of_birth             datetime            monthyear  \
0   Black/White    2014-04-10  2017-04-11 09:00:00  2017-04-11T09:00:00   
1   Brown/White    2013-11-18  2015-12-28 18:43:00  2015-12-28T18:43:00   
2     Red/White    2013-02-04  2016-02-11 12:41:00  2016-02-11T12:41:00   
3        Tortie    2013-09-01  2013-12-08 14:58:00  2013-12-08T14:58:00   
4  Silver Tabby    2015-05-02  2016-05-06 10:49:00  2016-05-06T10:49:00   